In [1]:
import magcolloids as mgc
import os
import sys
import numpy as np
import pandas as pd
import scipy

from multiprocessing import Pool
import tqdm.auto as tqdm

sys.path.insert(0, '../icenumerics')
import icenumerics as ice
sys.path.insert(0, '../Analysis')
from importlib import reload  
import chirality as chi
reload(chi)

import matplotlib.pyplot as plt
from matplotlib import patches

# Statistical Chirality study

    In this script, I will use the chirality.py file to compute the dataframe of the chirality of each frame of all the experiments. In order to then analyze the data in a statistical way.

In [2]:
data_directory = "/media/Carol/Magnetic/Simulations/Cairo_lattice_PROJECT/Output_simulations/Output_Script-8.1"

In [3]:
data_directory_1 = "/home/carolina/CairoLattice_ACI_PROJECT/Scripts"

In [4]:
df = pd.read_csv(os.path.join(data_directory_1,"Pentagon_coordinates_Script-9.2.dat"), sep = '\t',index_col = [0])

In [5]:
df['dx'] = np.NaN
df['dy'] = np.NaN
df['dz'] = np.NaN
df['Chi'] = np.NaN

In [6]:
def Chirality(exp_entry):
    
    e = exp_entry[1].e 
    l = exp_entry[1].l
    
    load_obj = mgc.trj_lazyread(os.path.join(data_directory,
                                             "ACI_CairoLattice_25mT-2500s_PeriodicBound_l%u_exp%u.lammpstrj"%(l,e)),
                                                output = ["x","y","z","mux","muy","muz"])    
    bnd = load_obj.get_bounds(slice(0,250000,50))
    trj = load_obj.read_trj(slice(0,250000,50))
    col_trj = ice.get_ice_trj(trj,bnd, atom_types = 1, trap_types = [2,3])
    
    Chirality = []
    
    for i in col_trj.index.get_level_values('frame').unique():
    
        pentagons = chi.take_spin_all_pentagons(col_trj, df, frame = i);
        Chirality.append(chi.chirality(pentagons));
        
    new_pentagons = pd.concat(Chirality, keys = col_trj.index.get_level_values('frame').unique())
    
    new_pentagons.to_csv(os.path.join(data_directory, "ACI_CairoLattice_25mT-2500s_PeriodicBound_chirality_l%u_exp%u.dat"%(l,e)), sep="\t")

    return;

In [7]:
e =  np.arange(10,50)
l =  [10]
L, E = np.meshgrid(l,e)
experiments = pd.DataFrame({"e":E.flatten(),"l":L.flatten()})
experiments

,e,l
0,10,10
1,11,10
2,12,10
3,13,10
4,14,10
5,15,10
6,16,10
7,17,10
8,18,10
9,19,10


In [8]:
%%time
if __name__ ==  '__main__': 
    num_processors = 5
    p=Pool(processes = num_processors)
        
    list(tqdm.tqdm(p.imap(Chirality,  experiments.iterrows()), total=len(experiments)));

/home/carolina/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/home/carolina/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/home/carolina/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va


CPU times: user 3.89 s, sys: 1.55 s, total: 5.44 s
Wall time: 1h 36min 36s
